In [946]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch.nn as nn
from __future__ import print_function
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [947]:
housing=pd.read_csv('Housing.csv')
housing.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [948]:
housing.mainroad = housing.mainroad.map(dict(yes=1,no=0))
housing.guestroom = housing.guestroom.map(dict(yes=1,no=0))
housing.basement = housing.basement.map(dict(yes=1,no=0))
housing.hotwaterheating = housing.hotwaterheating.map(dict(yes=1,no=0))
housing.airconditioning = housing.airconditioning.map(dict(yes=1,no=0))
housing.prefarea = housing.prefarea.map(dict(yes=1,no=0))

housing.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [949]:
housing.shape

(545, 13)

In [950]:
num_vars=['price', 'area','bathrooms','bedrooms', 'stories', 'parking']
data_set=housing[num_vars]
data_set.head()

,price,area,bathrooms,bedrooms,stories,parking
0,13300000,7420,2,4,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,2,3,2,2
3,12215000,7500,2,4,2,3
4,11410000,7420,1,4,2,2


In [951]:
data_set.shape

(545, 6)

In [952]:
scaler = MinMaxScaler()
data_set[num_vars]= scaler.fit_transform(data_set[num_vars])
data_set.head()

C:\Users\nrashvan\AppData\Local\Temp\1\ipykernel_16504\1593678094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set[num_vars]= scaler.fit_transform(data_set[num_vars])


,price,area,bathrooms,bedrooms,stories,parking
0,1.000000,0.396564,0.333333,0.6,0.666667,0.666667
1,0.909091,0.502405,1.000000,0.6,1.000000,1.000000
2,0.909091,0.571134,0.333333,0.4,0.333333,0.666667
3,0.906061,0.402062,0.333333,0.6,0.333333,1.000000
4,0.836364,0.396564,0.000000,0.6,0.333333,0.666667


In [953]:
X = data_set.iloc[:, 1:6].values
X.shape


(545, 5)

In [954]:
Y = data_set.iloc[:, 0].values
Y.shape

(545,)

In [955]:
Y =Y.reshape(-1,1)
Y.shape

(545, 1)

In [956]:
t_u=torch.tensor(X)
t_c=torch.tensor(Y)
t_c.shape

torch.Size([545, 1])

In [957]:
t_u.shape

torch.Size([545, 5])

In [958]:
n_samples= t_u.shape[0]
n_val= int(0.2 * n_samples)

shuffled_indices= torch.randperm(n_samples)

train_indices= shuffled_indices[:-n_val]
val_indices= shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([452, 178,  90,  92,   5, 223,  98, 503, 429, 240, 405, 351, 360, 500,
         125, 349, 196, 439, 136, 279, 542, 516, 247, 194, 303, 220,  51,  39,
         501,   9, 221,  72, 121, 266, 298, 334, 244, 338, 532, 493, 379,  34,
         462, 506, 400, 325,  54, 248, 509, 239, 151, 488, 394, 337, 273, 149,
         472, 296, 459, 300, 135, 146,  16, 132, 420,  38, 396, 260, 278,  84,
         478, 130, 186, 440, 252, 253, 232, 290, 384, 116, 475, 416, 359, 456,
         409, 203, 189, 101, 301, 191, 451, 508, 374,  42,  69,  40, 115, 207,
         293, 352,  76, 286, 147, 222, 184, 110, 117, 246, 218, 295, 377, 343,
         517, 525, 490, 407, 431, 314, 534, 373, 281, 454, 141,  71,  63, 378,
         272, 320, 539,  25, 109, 521,  55, 474,   1, 283, 254, 127, 463, 527,
         519, 161,  88, 241, 181,  57, 205, 450, 496,  43,  10, 172, 399,  32,
         211, 289, 413,  45, 120, 169, 263, 261,  37, 306, 368,  81, 345, 236,
         250, 510, 140, 299, 425, 144, 235, 212,  73

In [959]:
train_t_u= t_u[train_indices]
train_t_c=t_c[train_indices]

val_t_u= t_u[val_indices]
val_t_c= t_c[val_indices]


In [960]:
from collections import OrderedDict

In [961]:
seq_model=nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(5, 8)),
    ('hidden_activation', nn.Tanh()), 
    ('output_linear', nn.Linear(8, 1))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=5, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [962]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [963]:
seq_model.output_linear.bias

Parameter containing:
tensor([0.2115], requires_grad=True)

In [964]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1,n_epochs +1):
        
        t_p_train=model(train_t_u)
        loss_train= loss_fn(t_p_train, train_t_c)
        
        t_p_val=model(val_t_u)
        loss_val= loss_fn(t_p_val, val_t_c)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch ==1 or epoch % 20==0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}, "
                  f" Validation loss {loss_val.item():.4f}")

In [965]:
train_t_u = train_t_u.float()

In [966]:
print(train_t_u.dtype)

torch.float32


In [967]:
val_t_u=val_t_u.float()
print(val_t_u.dtype)

torch.float32


In [968]:
train_t_c=train_t_c.float()
print(train_t_c.dtype)

torch.float32


In [969]:
val_t_c=val_t_c.float()
print(val_t_c.dtype)

torch.float32


In [973]:
optimizer= optim.SGD(seq_model.parameters(), lr=1e-1)
training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
   train_t_u=train_t_u,
   val_t_u= val_t_u,
     train_t_c= train_t_c,
   val_t_c= val_t_c)

print('output', seq_model(val_t_u))
print('answer',  val_t_c)
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 0.0200,  Validation loss 0.0224
Epoch 20, Training loss 0.0171,  Validation loss 0.0186
Epoch 40, Training loss 0.0155,  Validation loss 0.0164
Epoch 60, Training loss 0.0146,  Validation loss 0.0151
Epoch 80, Training loss 0.0140,  Validation loss 0.0144
Epoch 100, Training loss 0.0137,  Validation loss 0.0139
Epoch 120, Training loss 0.0135,  Validation loss 0.0136
Epoch 140, Training loss 0.0133,  Validation loss 0.0134
Epoch 160, Training loss 0.0131,  Validation loss 0.0132
Epoch 180, Training loss 0.0130,  Validation loss 0.0130
Epoch 200, Training loss 0.0129,  Validation loss 0.0129
output tensor([[0.3432],
        [0.2204],
        [0.1677],
        [0.4481],
        [0.1378],
        [0.2278],
        [0.3936],
        [0.1006],
        [0.2240],
        [0.1117],
        [0.2786],
        [0.0957],
        [0.4803],
        [0.3205],
        [0.3561],
        [0.2913],
        [0.3295],
        [0.2743],
        [0.2875],
        [0.3691],
        [0.2